In [1]:
import sys
sys.path.append(sys.path[0]+'/../')

import torch
import torch.nn as nn
import time
from tqdm import tqdm
from torchsummary import summary
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from dataloaders.minc_dataloader import MINCDataset, MINCDataLoader
from dataloaders.irh_dataloader import IRHDataset

from utils.loss_resnet import LossRN

from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from torchvision.models import swin_t, Swin_T_Weights, swin_s, Swin_S_Weights, swin_b, Swin_B_Weights, swin_v2_s, Swin_V2_S_Weights


In [2]:
minc_path = '/home/ahmed/workspace/notebook/matrec/datasets/minc'
labels_path = '/home/ahmed/workspace/notebook/matrec/datasets/minc/train.txt'
labels_path_t = '/home/ahmed/workspace/notebook/matrec/datasets/minc/test.txt'

irh_path = "../datasets/irh/img_raw"
label_irh = "../datasets/irh/files/dataset.csv"

BATCH_SIZE = 6
TRAIN_ITER = 10000
TEST_ITER = 400
LOAD = True
start = 150000
lr = 0.00001
size = 300

#train_dataloader = MINCDataLoader(minc_path, labels_path, batch_size=BATCH_SIZE, size=size, f=0.16)

dataset_test = MINCDataset(minc_path, labels_path_t, size=(size, size))
test_loader = DataLoader(dataset=dataset_test, batch_size=BATCH_SIZE, num_workers=0, pin_memory=False, shuffle=True)


dataset_irh = IRHDataset(irh_path, label_irh, size=(size, size))
dataloader_irh = DataLoader(dataset=dataset_irh, batch_size=BATCH_SIZE, num_workers=BATCH_SIZE, pin_memory=False, shuffle=True)


In [3]:
from torchvision.models import resnet50

checkpoint = "../weights/resnet50_irh_.pth"

model = resnet50()
model.fc = nn.Linear(model.fc.in_features, 15)
model.load_state_dict(torch.load(checkpoint), strict=False)
"""for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True"""
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss = LossRN()

In [4]:
#add augmentation
#rescaling [0, 1] and normalization (mean, std)
#10+10+50+130
for epc in range(130):
    r = tqdm(dataloader_irh, leave=False, desc="Epoch {}: ".format(epc+1), total=len(dataloader_irh))    
    
    for x, y in r:
        y_pred = model(x.cuda())
        lf = loss.compute(y_pred, y)
        lf.backward()
        optimizer.step()
        optimizer.zero_grad()
        r.set_postfix(loss=lf.item())
torch.save(model.state_dict(), checkpoint)
print("done!!")

KeyboardInterrupt: 